# Thai COVID Data by Province?

Total by province: https://covid19.th-stat.com/api/open/cases/sum

Open Cases (JSON):  https://covid19.th-stat.com/api/open/cases

Most comprehensive but doesn't seem to be in English:  https://opend.data.go.th/get-ckan/datastore_search?resource_id=24ac8406-0cf9-4f8e-a55e-b53cf6766d1a&limit=5



In [1]:
#import libraries
import os, sys
import json
import pandas as pd
# import geopandas as gpd
import numpy as np
from numpy import int64
import requests, io
import urllib.request
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import logging
import xml.etree.ElementTree as ET
import datetime

sys.path.append('..')
from util.rss import RSSParser




logging.basicConfig(level=logging.DEBUG)

In [5]:
# get homepage
url = ' https://covid19.th-stat.com/api/open/cases'
response = requests.get(url)
data = response.content.decode('utf-8')
# soup = BeautifulSoup(response.text, 'lxml')

len(data) // number of characters

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): covid19.th-stat.com:443
DEBUG:urllib3.connectionpool:https://covid19.th-stat.com:443 "GET /api/open/cases HTTP/1.1" 200 None


3472987

In [60]:
pd.read_json(data).LastData.max()

'2021-01-08 20:32:07'

In [47]:
today_date = datetime.date.today()
today_datetime = datetime.datetime(today_date.year, today_date.month, today_date.day)
# today = datetime.date(2020, 1, 1)
print(f'today_date: {today_date}')
print(f'today_datetime: {today_datetime}')
# yesterday = datetime.date(today.year, today.month, today.day - 1)
yesterday = today_datetime - datetime.timedelta(days=1)
print(f'yesterday: {yesterday}')


# datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)

today_date: 2021-01-09
today_datetime: 2021-01-09 00:00:00
yesterday: 2021-01-08 00:00:00


In [36]:
parsedJson = json.loads(data)
df = pd.DataFrame.from_dict(parsedJson['Data'])
# convert confirmDate to datetime column
df['ConfirmDate'] = df['ConfirmDate'].astype('datetime64[ns]')
# df['ConfirmDate'] = pd.to_datetime(['ConfirmDate'])
# type(parsedJson)

In [20]:
df.columns

Index(['ConfirmDate', 'No', 'Age', 'Gender', 'GenderEn', 'Nation', 'NationEn',
       'Province', 'ProvinceId', 'District', 'ProvinceEn', 'Detail',
       'StatQuarantine'],
      dtype='object')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9869 entries, 0 to 9868
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ConfirmDate     9869 non-null   datetime64[ns]
 1   No              9868 non-null   object        
 2   Age             7286 non-null   float64       
 3   Gender          9869 non-null   object        
 4   GenderEn        9869 non-null   object        
 5   Nation          9869 non-null   object        
 6   NationEn        9805 non-null   object        
 7   Province        9869 non-null   object        
 8   ProvinceId      9869 non-null   int64         
 9   District        9869 non-null   object        
 10  ProvinceEn      9869 non-null   object        
 11  Detail          0 non-null      object        
 12  StatQuarantine  9869 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory usage: 1002.4+ KB


In [32]:
df.iloc[0]

ConfirmDate       2021-01-08 20:32:07
No                               9841
Age                               NaN
Gender                     ไม่ระบุเพศ
GenderEn                      Unknown
Nation                        ไม่ทราบ
NationEn                      Unknown
Province                    สมุทรสาคร
ProvinceId                         62
District                             
ProvinceEn               Samut Sakhon
Detail                           None
StatQuarantine                      1
Name: 0, dtype: object

In [19]:
df.ProvinceEn.unique()

array(['Samut Sakhon', 'Rayong', 'Chanthaburi', 'Samut Prakan',
       'Chonburi', 'Bangkok', 'Samut Songkhram', 'Chiang Mai', 'Lopburi',
       'Buriram', 'Surat Thani', 'Suphan Buri', 'Loei', 'Ang Thong',
       'Lamphun', 'Nakhon Pathom', 'Pathum Thani', 'Unknown', 'Trat',
       'Tak', 'Phra Nakhon Si Ayutthaya', 'Sing Buri', 'Ratchaburi',
       'Khon Kaen', 'Chachoengsao', 'Prachinburi', 'Surin', 'Narathiwat',
       'Phichit', 'Nakhon Ratchasima', 'Kanchanaburi', 'Nonthaburi',
       'Nan', 'Phetchaburi', 'Prachuap Khiri Khan', 'Chainat', 'Lampang',
       'Amnat Charoen', 'Sa Kaeo', 'Nakhon Si Thammarat',
       'Ubon Ratchathani', 'Nong Khai', 'Maha Sarakham', 'Nakhon Nayok',
       'Saraburi', 'Ranong', 'Uttaradit', 'Chiang Rai', 'Sukhothai',
       'Nakhon Sawan', 'Krabi', 'Satun', 'Chaiyaphum', 'Trang',
       'Songkhla', 'Phuket', 'Udon Thani', 'Kamphaeng Phet', 'Phetchabun',
       'Yala', 'Phayao', 'Pattani', 'Chumphon', 'Phang Nga',
       'Nakhon Phanom', 'Phatthalung'

In [33]:
df[df.ProvinceEn == "Bangkok"].District.unique()

array(['บางขุนเทียน', 'บางกะปิ', 'เมืองสมุทรสาคร', 'บางเขน', 'บางพลัด',
       'บางคอแหลม', 'บางพลี', 'สวนหลวง', 'บางละมุง', 'เมือง', 'ประเวศ',
       'บางนา', 'บางกรวย', 'ธนบุรี', 'บางแค', 'อัมพวา', 'เสนา',
       'บางบัวทอง', 'ดอนเมือง', 'บางบ่อ', 'วัฒนา', 'คลองเตย', 'คลองสาน',
       'บางซื่อ', 'คลองสามวา', 'ภาษีเจริญ', 'บางรัก', 'บ้านกุ่ม',
       'จอมทอง', 'หนองแขม', 'ปทุมวัน', 'บางใหญ่', 'บางเสาธง', 'ยานนาวา',
       'บางกอกใหญ่', 'ราษฎร์บูรณะ', 'ปากเกร็ด', 'ศรีขระภูมิ', 'จตุจักร',
       'มีนบุรี', 'ดุสิต', 'จอมบึง', 'พญาไท', 'ทวีวัฒนา', '', 'พระนคร',
       'สาทร', 'ห้วยขวาง', 'กระทุ่มแบน', 'วััฒนา', 'บางพลีัด',
       'ป้อมปราการศัตรูพ่าย', 'วังทองหลาง', 'พระโขนง', 'ดินแดง', 'บางนอน',
       'ลาดกระบัง', 'บางบอน', 'ทุ่งครุ', 'หลักสี่', 'รัชดา',
       'บางขุุนเทียน', 'บึงกุ่ม', '้เมือง', 'ยายนาวา', 'หนองจอก',
       'บางกอกน้อย', 'แก้งคร้อ', 'บ้านโป่ง', 'สาธร', 'คลองหลวง',
       'ป้อมปราบศัตรูพ่าย', 'ราชเทวี', 'บางมด', 'บางยี่ขัน', 'ดุุสิต',
       'สายไหม', 'คลองเตยเหนือ', '

# Pathum Wan District

In [22]:
df[df.District == "ปทุมวัน"]

,ConfirmDate,No,Age,Gender,GenderEn,Nation,NationEn,Province,ProvinceId,District,ProvinceEn,Detail,StatQuarantine
530,2021-01-07 17:32:10,9288,49.0,ชาย,Male,Thailand,Thailand,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
536,2021-01-07 17:32:10,9283,15.0,หญิง,Female,Thailand,Thailand,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
546,2021-01-07 17:32:10,9273,51.0,หญิง,Female,Thailand,Thailand,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
2880,2021-01-05 15:41:06,6893,51.0,ชาย,Male,Thailand,Thailand,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
4362,2021-01-01 18:55:46,5479,60.0,ไม่ระบุเพศ,Unknown,Thailand,Thailand,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
5149,2020-12-28 15:05:37,4706,50.0,ชาย,Male,Yemen,Yemen,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
5607,2020-12-23 10:49:03,4261,32.0,ชาย,Male,Brazil,Brazil,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
5614,2020-12-23 10:49:03,4246,38.0,ชาย,Male,Netherlands,Netherlands,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
5617,2020-12-23 10:49:03,4243,57.0,หญิง,Female,Switzerland,Swiss,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1
5650,2020-12-23 10:49:03,4210,51.0,ชาย,Male,Jordan,None,กรุงเทพมหานคร,1,ปทุมวัน,Bangkok,None,1


## Yesterday THA new case count

In [51]:
# confirmed that this is new cases for 1/8/2021
len(df[df.ConfirmDate > yesterday])

205

## Yesterday Bangkok new case count

In [54]:
df[(df.ConfirmDate > yesterday) & (df.District == "ปทุมวัน")]

,ConfirmDate,No,Age,Gender,GenderEn,Nation,NationEn,Province,ProvinceId,District,ProvinceEn,Detail,StatQuarantine


In [56]:
len(df[(df.ConfirmDate > yesterday) & (df.ProvinceEn == "Bangkok")])

46